*This is a simple backtesting of a trading strategy,i don't tend to advice any markets related advice*\
**SMA Crossover** is simply a technical trading strategy based on two moving averages one long term(200d) and another short term (50d)

The strategy works as
  * Buy Signal (Golden Cross):
    * Occurs when the short-term SMA crosses above the long-term SMA(cuts from below) Suggests upward momentum (bullish signal)

  * Sell Signal (Death Cross):
    * Occurs when the short-term SMA crosses below the long-term SMA(cuts from above) Suggests downward momentum (bearish signal)

disclaimer: lagging indicators as they rely on historical data


In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots # Corrected import
import numpy as np
!pip install backtesting
from backtesting import Backtest,Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 6.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [3]:
#creating dyanamic inputs for the user to choose stocks(tickers)
tickers = input('Enter the ticker for the stock you want to analyze: ')
raw_data= yf.Ticker(tickers)
price_data=raw_data.history(interval='1h',period='200d')
price_data= price_data.iloc[:-1]#removing the last row of data

Enter the ticker for the stock you want to analyze: GRSE.NS


I have used **Garden Reach Shipbuilders & Enginers Ltd(GRSE.NS)** as it is one of the most volatile stock, so i wanted to see the benefits of the same\
although, High Volatality doesn't really mean price trend
  * for example- if the market is choppy(the stock price is oving up-down,up-down) it will lead to many false buy indication

In [4]:
# Defining a function that calculates SMA
def simple_moving_average(price, period=30):
    return pd.Series(price).rolling(window=period).mean()

class SMA_Crossover(Strategy):
    interval_short_term = 50
    interval_long_term = 200

    def init(self):
        self.sma_short_term = self.I(simple_moving_average, self.data.Close, self.interval_short_term)
        self.sma_long_term = self.I(simple_moving_average, self.data.Close, self.interval_long_term)

    def next(self):
        if crossover(self.sma_short_term, self.sma_long_term):
            self.buy()
        elif crossover(self.sma_long_term, self.sma_short_term):
            if self.position:
                self.position.close()  # Focusing on buy-only strategy

# Assuming price_data is a Pandas DataFrame with at least a 'Close' column
backtest = Backtest(price_data, SMA_Crossover, cash=100000, commission=0.002, exclusive_orders=True)
output= pd.DataFrame(backtest.run())
print(output)


Backtest.run:   0%|          | 0/1185 [00:00<?, ?bar/s]

                                                                        0
Start                                           2024-10-04 09:15:00+05:30
End                                             2025-07-23 14:15:00+05:30
Duration                                                292 days 05:00:00
Exposure Time [%]                                               45.848375
Equity Final [$]                                            162074.116483
Equity Peak [$]                                             191999.272251
Commissions [$]                                               1249.337497
Return [%]                                                      62.074116
Buy & Hold Return [%]                                            86.27727
Return (Ann.) [%]                                               84.887818
Volatility (Ann.) [%]                                           85.130022
CAGR [%]                                                        51.654998
Sharpe Ratio                          

**Explaining some terms**\
*Shortino ratio* only focuses on the downside deviation whereas sharpe ratio accounts for total volatality
  * for example, *sharpe ratio* when accounts for total volatality it also accounts for the massive profits(if any), which makes this measure unreliable for knowing actual risk\
  in the case of *shortino ratio* it only accounts for the downside volatality(or target) this gives a much clear view of the actual risk of the portfolio

*Calmar Ratio* is the ratio which accounts for the maximum loss(as in maximum dip it took) to prepare for the unexpected losses, this ratio is for the long terms use cases where the investor wants to feel safe and aware of the potential dip.\
*Alpha* is simply the unexplained returns, which are like deviations from the expected returns\
*Max draw-downs* this is basically the percentage chnage from the top to bottom, if out portoflio was at INR 1000 and it dropped down to 700 then our max draw down would be -30%(this is always negative) for the period it rises again\



In [5]:
backtest.plot()

/tmp/ipython-input-5-3521804055.py:1: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  backtest.plot()


/usr/local/lib/python3.11/dist-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p1327', ...)

**Explanation of the above plot:**
  *  *Equity Pannel* is the top section which shows our portfolio value over time\
    * It shows peak value,ending value and the most important it shows how long our portolfio was under the water,i.e, in our case our portfolio was under the water(max D.D) for 158 days, this is a very long period and it may stress the investor and he/she may close their positions due to the stress

  *  *Profit and loss* this section tells us when did we close our positions. The red dots represents the times when we closed our position, red dots represents loss and green dot represnts profits

  *  *The main pannel* summarizes the whole trading cycle about when did we short/long when did we eneter/exited the position, this is a key pannel to understand how our trade performed, on every trade.

  *  *Volume* is basically the ammount of trades, if the volume increases means big players are taking a postion in the market, hence be aware as there might be a rally.

In [8]:
#Define RSI
#RSI clalculation function
def RSI_calculate(close,lookback):
  delta=close.diff()
  up=[]
  down=[]
  for i in range(len(delta)):
    if delta[i]<0:
      up.append(0)
      down.append(delta[i])
    else:
      up.append(delta[i])
      down.append(0)
  up_series=pd.Series(up)
  down_series=pd.Series(down).abs()

  up_ewm=up_series.ewm(com=lookback -1,adjust=False).mean()
  down_ewm=down_series.ewm(com=lookback -1,adjust=False).mean()

  rs=up_ewm/down_ewm
  rsi= 100 -(100/(1+rs))
  rsi_df=pd.DataFrame(rsi).rename(columns={0:'rsi'}).set_index(close.index)
  rsi_df=rsi_df.dropna()
  return rsi_df[3:]
price_data['RSI']=RSI_calculate(price_data['Close'],14)
price_data=price_data.dropna()
price_data=price_data.iloc[14:]#make sure to skip 14 days to have real values
price_data

/tmp/ipython-input-8-680766953.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipython-input-8-680766953.py:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipython-input-8-680766953.py:10: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI
2024-10-11 10:15:00+05:30,1758.599976,1765.750000,1735.000000,1744.199951,239938,0.0,0.0,74.760139
2024-10-11 11:15:00+05:30,1744.250000,1747.699951,1737.000000,1740.949951,73909,0.0,0.0,73.435156
2024-10-11 12:15:00+05:30,1740.199951,1759.550049,1740.150024,1752.000000,80777,0.0,0.0,75.054006
2024-10-11 13:15:00+05:30,1750.250000,1751.500000,1741.000000,1746.250000,45640,0.0,0.0,72.575567
2024-10-11 14:15:00+05:30,1747.300049,1754.500000,1742.500000,1748.000000,87525,0.0,0.0,72.869212
...,...,...,...,...,...,...,...,...
2025-07-23 10:15:00+05:30,2632.000000,2632.000000,2610.000000,2618.300049,64559,0.0,0.0,47.262226
2025-07-23 11:15:00+05:30,2616.600098,2631.100098,2610.000000,2623.300049,30769,0.0,0.0,49.028722
2025-07-23 12:15:00+05:30,2623.300049,2659.899902,2621.100098,2632.199951,125182,0.0,0.0,52.104037
2025-07-23 13:15:00+05:30,2633.000000,2645.800049,2630.199951,2645.000000,36831,0.0,0.0,56.197355


In [7]:
#plot rsi
fig=go.Figure(make_subplots(rows=1,cols=1,shared_xaxes=True,vertical_spacing=0.05))
fig.add_trace(go.Scatter(x=price_data.index,y=price_data['RSI'],name='RSI',marker_color='grey'))

fig.add_trace(go.Scatter(x=price_data.index,y=[30]*len(price_data.index),name='oversold',marker_color='green'))
fig.add_trace(go.Scatter(x=price_data.index,y=[70]*len(price_data.index),name='overbought',marker_color='red'))

fig.update_layout(title='RSI Plot', xaxis_title='Date', yaxis_title='RSI Value',template='plotly_dark')
fig.show()

**RSI(Relative Strength Index)** is bascially telling us the average day up/average day downs\
this is used to understand when was the stock over sold or over bought, it can help us predict what might be the next movement in the markets and should we short or long\
for example- on march 20,2025 the stock was having an RSI of 90.41, and then the stock dropped because it was over bought. this works vice-versa

**Conclusion** can be derived, that is, SMA cross over is not a solid strategy, as we can see that we incurred losses on 27th december and 11th feb, although the losses were small and one may think that the proffits we incurred later on were much larger\
the thing to understand is, that no technichal strategy garuntees us profits and one should not rely on them for ever.